In [1]:
import tensorflow as tf
import os
from six.moves import cPickle
import collections
import numpy as np
import codecs

### 读取和预处理数据

请打开terminal并cd到当前notebook所在目录，执行下列命令
```
mkdir data
cd data
mkdir sherlock
cd sherlock
wget https://sherlock-holm.es/stories/plain-text/cnus.txt
mv cnus.txt input.txt
```

In [5]:
data_dir = "./data/sherlock/"


input_file = os.path.join(data_dir, "input.txt")
vocab_file = os.path.join(data_dir, "vocab.pkl")
tensor_file = os.path.join(data_dir, "data.npy")
    

with codecs.open(input_file, "r", encoding="utf-8") as f:
    data = f.read()
counter = collections.Counter(data)
counter_pairs = sorted(counter.items(), key=lambda x: -x[1])
chars, _ = zip(*counter_pairs)
vocab_size = len(chars)
vocab = dict(zip(chars, range(len(chars))))
tensor = np.array(list(map(vocab.get, data)))
    

### 把数据处理成batch

In [6]:
batch_size = 50
seq_length = 50
num_batches = int(tensor.size / (batch_size * seq_length))
if num_batches == 0:
    assert False, "Not enough data. Make seq_length and batch_size small."
tensor = tensor[:num_batches * batch_size * seq_length]
xdata = tensor
ydata = np.copy(tensor)
ydata[:-1] = xdata[1:]
ydata[-1] = xdata[0]
x_batches = np.split(xdata.reshape(batch_size, -1), num_batches, 1)
y_batches = np.split(ydata.reshape(batch_size, -1), num_batches, 1)
pointer = 0

### 定义RNN模型

In [7]:
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq


class Model():
    def __init__(self, training=True):
        self.batch_size = 50
        self.seq_length = 50
        if not training:
            self.batch_size = 1
            self.seq_length = 1
            
        self.rnn_size = 128
        self.num_layers = 2
        self.input_keep_prob = 1.0
        self.output_keep_prob = 1.0
        self.grad_clip = 5.0
        self.training = 1

        cell_fn = rnn.BasicRNNCell

        cells = []
        for _ in range(self.num_layers):
            cell = cell_fn(self.rnn_size)
            # dropout
            if training and (self.input_keep_prob < 1.0 or self.output_keep_prob < 1.0):
                cell = rnn.DropoutWrapper(cell, input_keep_prob=self.input_keep_prob,
                                         output_keep_prob=self.output_keep_prob)
            cells.append(cell)

        self.cell = cell = rnn.MultiRNNCell(cells, state_is_tuple=True)

        # placeholder for input and output
        self.input_data = tf.placeholder(tf.int32, [self.batch_size, self.seq_length])
        self.output_data = tf.placeholder(tf.int32, [self.batch_size, self.seq_length])
        self.initial_state = cell.zero_state(self.batch_size, tf.float32)
        with tf.variable_scope("rnnlm"):
            softmax_w = tf.get_variable("softmax_w", [self.rnn_size, vocab_size])
            softmax_b = tf.get_variable("softmax_b", [vocab_size])
        embedding = tf.get_variable("embedding", [vocab_size, self.rnn_size])
        inputs = tf.nn.embedding_lookup(embedding, self.input_data)
        if training and self.output_keep_prob:
            inputs = tf.nn.dropout(inputs, self.output_keep_prob)

        inputs = tf.split(inputs, self.seq_length, 1)
        inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

        def loop(prev, _):
            prev = tf.matmul(prev, softmax_w) + softmax_b
            prev_symbol = tf.stop_gradient(tf.argmax(prev, 1))
            return tf.nn.embedding_lookup(embedding, prev_symbol)

        outputs, last_state = legacy_seq2seq.rnn_decoder(inputs, self.initial_state, cell, 
                                                         loop_function=loop if not training else None, scope="rnnlm")
        output = tf.reshape(tf.concat(outputs, 1), [-1, self.rnn_size])
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        self.probs = tf.nn.softmax(self.logits)
        loss = legacy_seq2seq.sequence_loss_by_example([self.logits],
                                                      [tf.reshape(self.output_data, [-1])],
                                                      [tf.ones([self.batch_size * self.seq_length])])
        with tf.name_scope("cost"):
            self.cost = tf.reduce_sum(loss) / batch_size / seq_length
        self.final_state = last_state
        self.lr = tf.Variable(0.0, trainable=False)
        # gradient clipping on trainable variables
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), self.grad_clip)

        # optimizer and train
        with tf.name_scope("optimizer"):
            optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
    
    def sample(self, sess, chars, vocab, num=200, prime="The "):
        state = sess.run(self.cell.zero_state(1, tf.float32))
        for char in prime[:-1]:
            x = np.zeros((1,1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state: state}
            [state] = sess.run([self.final_state], feed)

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return (int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = prime
        char = prime[-1]
        for n in range(num):
            x = np.zeros((1,1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state: state}
            [p, state] = sess.run([self.probs, self.final_state], feed)
            p = p[0]
            sample = weighted_pick(p)

            pred = chars[sample]
            ret += pred
            char = pred
        return ret



### 开始训练RNN

In [8]:
import time
num_epochs = 10
learning_rate = 0.002
decay_rate = 0.97
save_dir = "./save/"
save_every = 500


tf.reset_default_graph()
model = Model() 


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())   
    saver = tf.train.Saver(tf.global_variables())

    for e in range(num_epochs):
        sess.run(tf.assign(model.lr, learning_rate*(decay_rate ** e)))
        # reset the pointer to load from the beginning
        pointer = 0
        state = sess.run(model.initial_state)
        for b in range(num_batches):
            start = time.time()
            x, y = x_batches[pointer], y_batches[pointer]
            pointer += 1
            feed = {model.input_data: x, model.output_data: y}
            feed[model.initial_state] = state
                
            train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
            end = time.time()
            print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch={:.3f}".format(
                e * num_batches + b, num_epochs * num_batches, e, train_loss, end-start))
            
            if ( e * num_batches + b ) % save_every == 0 or ( 
                e == num_epochs - 1 and b == num_batches - 1):
                checkpoint_path = os.path.join(save_dir, "model.ckpt")
                saver.save(sess, checkpoint_path, global_step=e*num_batches + b)
                print("model saved to {}".format(checkpoint_path))
                
        
        

0/67600 (epoch 0), train_loss = 4.606, time/batch=0.441
model saved to ./save/model.ckpt
1/67600 (epoch 0), train_loss = 4.428, time/batch=0.073
2/67600 (epoch 0), train_loss = 4.092, time/batch=0.075
3/67600 (epoch 0), train_loss = 3.728, time/batch=0.073
4/67600 (epoch 0), train_loss = 3.528, time/batch=0.147
5/67600 (epoch 0), train_loss = 3.380, time/batch=0.169
6/67600 (epoch 0), train_loss = 3.286, time/batch=0.093
7/67600 (epoch 0), train_loss = 3.208, time/batch=0.069
8/67600 (epoch 0), train_loss = 3.147, time/batch=0.072
9/67600 (epoch 0), train_loss = 3.075, time/batch=0.076
10/67600 (epoch 0), train_loss = 3.015, time/batch=0.082
11/67600 (epoch 0), train_loss = 3.030, time/batch=0.072
12/67600 (epoch 0), train_loss = 3.025, time/batch=0.071
13/67600 (epoch 0), train_loss = 3.005, time/batch=0.079
14/67600 (epoch 0), train_loss = 3.018, time/batch=0.073
15/67600 (epoch 0), train_loss = 2.967, time/batch=0.069
16/67600 (epoch 0), train_loss = 3.000, time/batch=0.194
17/67600

145/67600 (epoch 0), train_loss = 2.063, time/batch=0.071
146/67600 (epoch 0), train_loss = 2.001, time/batch=0.075
147/67600 (epoch 0), train_loss = 2.009, time/batch=0.070
148/67600 (epoch 0), train_loss = 2.052, time/batch=0.117
149/67600 (epoch 0), train_loss = 1.935, time/batch=0.071
150/67600 (epoch 0), train_loss = 2.010, time/batch=0.071
151/67600 (epoch 0), train_loss = 1.979, time/batch=0.073
152/67600 (epoch 0), train_loss = 1.963, time/batch=0.069
153/67600 (epoch 0), train_loss = 2.025, time/batch=0.075
154/67600 (epoch 0), train_loss = 1.948, time/batch=0.073
155/67600 (epoch 0), train_loss = 2.017, time/batch=0.069
156/67600 (epoch 0), train_loss = 2.032, time/batch=0.069
157/67600 (epoch 0), train_loss = 2.016, time/batch=0.077
158/67600 (epoch 0), train_loss = 1.942, time/batch=0.233
159/67600 (epoch 0), train_loss = 1.999, time/batch=0.073
160/67600 (epoch 0), train_loss = 1.977, time/batch=0.088
161/67600 (epoch 0), train_loss = 1.915, time/batch=0.068
162/67600 (epo

288/67600 (epoch 0), train_loss = 1.800, time/batch=0.071
289/67600 (epoch 0), train_loss = 1.878, time/batch=0.071
290/67600 (epoch 0), train_loss = 1.817, time/batch=0.085
291/67600 (epoch 0), train_loss = 1.781, time/batch=0.072
292/67600 (epoch 0), train_loss = 1.799, time/batch=0.070
293/67600 (epoch 0), train_loss = 1.804, time/batch=0.189
294/67600 (epoch 0), train_loss = 1.828, time/batch=0.100
295/67600 (epoch 0), train_loss = 1.823, time/batch=0.085
296/67600 (epoch 0), train_loss = 1.785, time/batch=0.068
297/67600 (epoch 0), train_loss = 1.775, time/batch=0.071
298/67600 (epoch 0), train_loss = 1.757, time/batch=0.074
299/67600 (epoch 0), train_loss = 1.835, time/batch=0.070
300/67600 (epoch 0), train_loss = 1.779, time/batch=0.070
301/67600 (epoch 0), train_loss = 1.815, time/batch=0.073
302/67600 (epoch 0), train_loss = 1.792, time/batch=0.073
303/67600 (epoch 0), train_loss = 1.768, time/batch=0.069
304/67600 (epoch 0), train_loss = 1.820, time/batch=0.072
305/67600 (epo

431/67600 (epoch 0), train_loss = 1.684, time/batch=0.087
432/67600 (epoch 0), train_loss = 1.685, time/batch=0.071
433/67600 (epoch 0), train_loss = 1.687, time/batch=0.080
434/67600 (epoch 0), train_loss = 1.677, time/batch=0.075
435/67600 (epoch 0), train_loss = 1.768, time/batch=0.070
436/67600 (epoch 0), train_loss = 1.717, time/batch=0.080
437/67600 (epoch 0), train_loss = 1.693, time/batch=0.094
438/67600 (epoch 0), train_loss = 1.714, time/batch=0.221
439/67600 (epoch 0), train_loss = 1.719, time/batch=0.069
440/67600 (epoch 0), train_loss = 1.677, time/batch=0.111
441/67600 (epoch 0), train_loss = 1.691, time/batch=0.071
442/67600 (epoch 0), train_loss = 1.662, time/batch=0.072
443/67600 (epoch 0), train_loss = 1.668, time/batch=0.133
444/67600 (epoch 0), train_loss = 1.667, time/batch=0.108
445/67600 (epoch 0), train_loss = 1.743, time/batch=0.072
446/67600 (epoch 0), train_loss = 1.670, time/batch=0.079
447/67600 (epoch 0), train_loss = 1.643, time/batch=0.076
448/67600 (epo

574/67600 (epoch 0), train_loss = 1.617, time/batch=0.072
575/67600 (epoch 0), train_loss = 1.604, time/batch=0.070
576/67600 (epoch 0), train_loss = 1.642, time/batch=0.125
577/67600 (epoch 0), train_loss = 1.635, time/batch=0.072
578/67600 (epoch 0), train_loss = 1.683, time/batch=0.070
579/67600 (epoch 0), train_loss = 1.617, time/batch=0.071
580/67600 (epoch 0), train_loss = 1.587, time/batch=0.071
581/67600 (epoch 0), train_loss = 1.621, time/batch=0.071
582/67600 (epoch 0), train_loss = 1.579, time/batch=0.068
583/67600 (epoch 0), train_loss = 1.565, time/batch=0.073
584/67600 (epoch 0), train_loss = 1.630, time/batch=0.069
585/67600 (epoch 0), train_loss = 1.660, time/batch=0.071
586/67600 (epoch 0), train_loss = 1.575, time/batch=0.221
587/67600 (epoch 0), train_loss = 1.680, time/batch=0.088
588/67600 (epoch 0), train_loss = 1.578, time/batch=0.093
589/67600 (epoch 0), train_loss = 1.596, time/batch=0.073
590/67600 (epoch 0), train_loss = 1.662, time/batch=0.070
591/67600 (epo

...

22059/67600 (epoch 16), train_loss = 1.223, time/batch=0.117
22060/67600 (epoch 16), train_loss = 1.143, time/batch=0.132
22061/67600 (epoch 16), train_loss = 1.190, time/batch=0.121
22062/67600 (epoch 16), train_loss = 1.254, time/batch=0.279
22063/67600 (epoch 16), train_loss = 1.184, time/batch=0.135
22064/67600 (epoch 16), train_loss = 1.242, time/batch=0.128
22065/67600 (epoch 16), train_loss = 1.245, time/batch=0.119
22066/67600 (epoch 16), train_loss = 1.183, time/batch=0.125
22067/67600 (epoch 16), train_loss = 1.286, time/batch=0.099
22068/67600 (epoch 16), train_loss = 1.217, time/batch=0.127
22069/67600 (epoch 16), train_loss = 1.234, time/batch=0.257
22070/67600 (epoch 16), train_loss = 1.229, time/batch=0.137
22071/67600 (epoch 16), train_loss = 1.210, time/batch=0.113
22072/67600 (epoch 16), train_loss = 1.208, time/batch=0.126
22073/67600 (epoch 16), train_loss = 1.239, time/batch=0.117
22074/67600 (epoch 16), train_loss = 1.202, time/batch=0.130
22075/67600 (epoch 16), 

22194/67600 (epoch 16), train_loss = 1.165, time/batch=0.247
22195/67600 (epoch 16), train_loss = 1.210, time/batch=0.158
22196/67600 (epoch 16), train_loss = 1.183, time/batch=0.110
22197/67600 (epoch 16), train_loss = 1.202, time/batch=0.120
22198/67600 (epoch 16), train_loss = 1.215, time/batch=0.121
22199/67600 (epoch 16), train_loss = 1.238, time/batch=0.131
22200/67600 (epoch 16), train_loss = 1.229, time/batch=0.110
22201/67600 (epoch 16), train_loss = 1.193, time/batch=0.182
22202/67600 (epoch 16), train_loss = 1.176, time/batch=0.207
22203/67600 (epoch 16), train_loss = 1.178, time/batch=0.095
22204/67600 (epoch 16), train_loss = 1.201, time/batch=0.136
22205/67600 (epoch 16), train_loss = 1.208, time/batch=0.116
22206/67600 (epoch 16), train_loss = 1.226, time/batch=0.137
22207/67600 (epoch 16), train_loss = 1.183, time/batch=0.119
22208/67600 (epoch 16), train_loss = 1.236, time/batch=0.093
22209/67600 (epoch 16), train_loss = 1.216, time/batch=0.262
22210/67600 (epoch 16), 

22329/67600 (epoch 16), train_loss = 1.253, time/batch=0.125
22330/67600 (epoch 16), train_loss = 1.246, time/batch=0.229
22331/67600 (epoch 16), train_loss = 1.178, time/batch=0.171
22332/67600 (epoch 16), train_loss = 1.218, time/batch=0.110
22333/67600 (epoch 16), train_loss = 1.247, time/batch=0.171
22334/67600 (epoch 16), train_loss = 1.210, time/batch=0.118
22335/67600 (epoch 16), train_loss = 1.165, time/batch=0.121
22336/67600 (epoch 16), train_loss = 1.217, time/batch=0.110
22337/67600 (epoch 16), train_loss = 1.197, time/batch=0.127
22338/67600 (epoch 16), train_loss = 1.205, time/batch=0.119
22339/67600 (epoch 16), train_loss = 1.238, time/batch=0.278
22340/67600 (epoch 16), train_loss = 1.231, time/batch=0.116
22341/67600 (epoch 16), train_loss = 1.182, time/batch=0.136
22342/67600 (epoch 16), train_loss = 1.219, time/batch=0.125
22343/67600 (epoch 16), train_loss = 1.123, time/batch=0.181
22344/67600 (epoch 16), train_loss = 1.209, time/batch=0.127
22345/67600 (epoch 16), 

22464/67600 (epoch 16), train_loss = 1.212, time/batch=0.139
22465/67600 (epoch 16), train_loss = 1.220, time/batch=0.124
22466/67600 (epoch 16), train_loss = 1.188, time/batch=0.180
22467/67600 (epoch 16), train_loss = 1.257, time/batch=0.190
22468/67600 (epoch 16), train_loss = 1.121, time/batch=0.161
22469/67600 (epoch 16), train_loss = 1.239, time/batch=0.126
22470/67600 (epoch 16), train_loss = 1.157, time/batch=0.135
22471/67600 (epoch 16), train_loss = 1.209, time/batch=0.103
22472/67600 (epoch 16), train_loss = 1.268, time/batch=0.131
22473/67600 (epoch 16), train_loss = 1.228, time/batch=0.272
22474/67600 (epoch 16), train_loss = 1.223, time/batch=0.148
22475/67600 (epoch 16), train_loss = 1.219, time/batch=0.115
22476/67600 (epoch 16), train_loss = 1.214, time/batch=0.131
22477/67600 (epoch 16), train_loss = 1.200, time/batch=0.127
22478/67600 (epoch 16), train_loss = 1.198, time/batch=0.160
22479/67600 (epoch 16), train_loss = 1.199, time/batch=0.136
22480/67600 (epoch 16), 

KeyboardInterrupt: 

In [ ]:
saver.saver_def.filename_tensor_name

### 产生一些句子试一试

In [8]:
n = 500
prime = " "

tf.reset_default_graph()
model = Model(training=False)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state(save_dir)
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print(model.sample(sess, chars, vocab, n, prime).encode("utf-8"))


# with open(os.path.join(save_dir, "config.pkl"), "rb") as f:
#     saved_args = cPickle.load(f)
# with open(os.path.join(save_dir, "chars_vocab.pkl"), "rb") as f:
#     chars, vocab = cPickle.load(f)


INFO:tensorflow:Restoring parameters from ./save/model.ckpt-500
b' and interess."\n\n     "Louted-rooked to thone co we-with foo hervers the deadper, deroubn, for the tage.\n\n     "Boordinathere whot stanted an\n     nenlifer, frabobing and feong to ond\n     the\n     tell offred tushingow It rage wh\n     his musuppleds threr is gin come\'s the kevericelds, and a redorness, and a dees\n     rearse\n     pfors a saged and thrred to With on\n     the poring the wimed th he pater of mary he "What he glought upond upon cloopen ofred heriging up. To the strateen to the paces'
